In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Generación de DF</h4>

In [ ]:
ruta_bd = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"
ruta_capa = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\tbl_capa"

# Construcción del diccionario a partir de la tabla: Capa
df_capa = pd.DataFrame.spatial.from_table(ruta_capa)
df_capa_to_dict = df_capa.loc[:,['codigo_capa','nombre_capa']]
df_capa_to_dict = df_capa_to_dict.set_index('codigo_capa')
diccionario_capa = df_capa_to_dict.to_dict()

# Modelado del diccionario tradicional
diccionario_capas_definitivo = diccionario_capa['nombre_capa']

<h4>Actualización de cada capa -registros- con el -codigo_capa-</h4>

In [ ]:
arcpy.env.workspace = ruta_bd
nombre_campo_codigo_capa = 'codigo_capa'

capas_modelo = ['tbl_capa', 'tbl_departamento', 'tbl_tematica', 'tbl_direccion', 'municipio', 'cuenca']

for elemento in arcpy.ListFeatureClasses():
    for capa_a_no_actualizar in capas_modelo:
        if elemento != capa_a_no_actualizar:
            # Aquí se puede modificar el tipo de campo para el -codigo_capa-
            arcpy.management.AddField(elemento, nombre_campo_codigo_capa, 'LONG', field_alias = 'Codigo Capa')
            for llave, valor in diccionario_capas_definitivo.items():
                if elemento == valor:
                    arcpy.management.CalculateField(elemento, nombre_campo_codigo_capa, int(llave), 'PYTHON3')
                    print("Se actualiza el campo -codigo_capa-, para la capa {0}".format(valor))
        else:
            print("La capa no se actualiza al ser {0}".format(capa_a_no_actualizar))

<h4>Borrado de campos innecesarios, capas de registros</h4>

In [ ]:
campos_eliminar = ['nombre_capa','codigo_tematica','nombre_capa']
codigo_capa = "codigo_capa"

for elemento in arcpy.ListFeatureClasses():
    for columnas in arcpy.ListFields(elemento):
        for i in campos_eliminar:
            if i == columnas.name:
                arcpy.management.DeleteField(elemento, i)
                print("Se elimina la columna {0}, de la capa {1}".format(elemento, columnas.name))
                
for elemento in arcpy.ListFeatureClasses():
    if elemento == 'cuenca':
        if arcpy.ListFields(elemento, codigo_capa):
            arcpy.management.DeleteField(elemento, codigo_capa)
            print("Se elimina la columna {0}, de la capa {1}".format(codigo_capa, elemento))